This script calculates the toroidal magnetic flux to initialise the spheromak CME model, given the poloidal flux. Based on Scolini et al, 2019 and Scolini et al, 2020b.

Copyright: Anwesha Maharana (March 2021), KU Leuven, Belgium

NOAA SWPC data archive
ftp.swpc.noaa.gov/pub/warehouse

RibbonDB catalog: (for flare peak intensity values)
http://solarmuri.ssl.berkeley.edu/~kazachenko/RibbonDB/

I is the flare peak intensity passed as argument [W/m^2]. 
phi_r (from the empirical relations) is in Mx = 1e-8 Wb
v3D = projected speed from GCS

Parameters to change:
1. r0 contains the radius of spheromak.
2. I1 is the flare peak intensity.
3. v3D1 is the total speed of the CME.

In [3]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import math

In [6]:
I1=1e-6; v3D1=960 
#CME1 in 2021-11-01 event. Ribbon catalog not updated for 2021. Find out the latest.

# Empirical flux relations

In [8]:
#################################
# Kazachenko+2017 (Flare ribbons)
#################################
def phip_kaz(I):
	return 10**(24.42)*I**(0.64)

################################
#Tschernitz+2018 (Flare ribbons)
################################
def phip_tscher(I):
	return 10**(24.21)*I**(0.58)

####################################
#Dissauer+2018(a) (Coronal Dimmings)
####################################
def phip_dis(I):
	return 10**(23.26)*I**(0.42)	

##########################
#Pal+2018; Gopalswamy+2017
##########################
def phip_speed(v3D):
	#v3D is in [km/s] #phip is in the order of [10**21 Mx]
	return (v3D/327)**(1/0.69)

phi_r_kaz = phip_kaz(I1) 
phi_r_speed = phip_speed(v3D1)
print ('Unfiltered phip_kaz = ', phi_r_kaz)#, 10**(phi_r_kaz))
print ('Unfiltered phip_speed = ', phip_speed(v3D1))

Unfiltered phip_kaz =  3.8018939632056264e+20
Unfiltered phip_speed =  4.762775561060195


# Poloidal to toroidal for spheromak

In [9]:
rsun_to_m = 6.96e8 
def calc_B0(phip,rs,alpha1):
    #x = alpha*rs
    x = 2.4048
    alpha = 0.532/rs
    print ('alpha = ', alpha, ', alpha1 = ', alpha1, ', xs = ', x, ', rs = ',rs)
    return pow(alpha1,3)*phip*rs/(2.0*np.pi*(np.sin(x)-x*np.cos(x)))

def integrand(x):
    return math.sin(x)/x

def calc_phi_t(B0,alpha):    
    low = 0 
    up = x01
    I = quad(integrand, low, up, args=())
    return 2.0*B0*(-np.sin(x01) + I[0])/pow(alpha,2)
    
def pol2tor_spr(r0, phip):
    r0 = r0*rsun_to_m; rs = 0.532*r0 # magnetic field of spheromak is completely axial [Rsun]
    alpha = x01/r0
    xs = alpha*rs #2.4048 # first zero of J0
    alpha1 = xs/rs
    print("xs = ", xs, "alpha = ", alpha, "alpha1 = ", alpha1)
    B0 = calc_B0(phip,rs,alpha1) 
    phit = calc_phi_t(B0,alpha1)    
    return B0, phit

In [11]:
#-------------------------------------------------
#Constants and parameters required for computation
#-------------------------------------------------
hw=45
r0 = 20*math.tan(hw*np.pi/180) #21.41 # Radius of Spheromak [Rsun]
r1=14.0; r2=19.0; r3=21.0
Rtip = 21.5; L = 2.0*Rtip
x01 = 4.4934 # first zero of J1
x00 = 2.4048 # first zero of J0

In [16]:
#computing average phi_p
phip_arr1= [phip_kaz(I1)*1e-8, phip_tscher(I1)*1e-8, phip_dis(I1)*1e-8, phip_speed(v3D1)*1e21*1e-8]
print (phip_arr1)

phip_avg1 = np.average(phip_arr1)
print ("Average poloidal flux = ", np.format_float_scientific(phip_avg1, exp_digits=2))#, f"{phip_avg1:.2e}")

[3801893963205.6265, 5370317963702.541, 5495408738576.267, 47627755610601.945]
Average poloidal flux =  1.5573844069021594e+13


In [15]:
#converting poloidal to toroidal flux
print (r1,phip_avg1)
B0, phit = pol2tor_spr(r1, phip_avg1)
print ('B0 = ', np.format_float_scientific(B0, exp_digits=2), 'toroidal flux = ', np.format_float_scientific(phit, exp_digits=2))

14.0 15573844069021.594
xs =  2.3904888 alpha =  4.6114532019704434e-10 alpha1 =  4.6114532019704434e-10
alpha =  1.026272577996716e-10 , alpha1 =  4.6114532019704434e-10 , xs =  2.4048 , rs =  5183808000.0
B0 =  5.136707576754224e-07 toroidal flux =  1.2713766853300783e+13
